In [1]:
from path_arquivos import *
import pandas as pd
import numpy as np
import glob
import os
caminho = ar_xlsx.ar_96

In [3]:
df = pd.read_excel(caminho)
display(df.dtypes)
display(df.columns)

c:\WS_OLIVEIRA\SCRIPTS\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


CODFILIAL              int64
CODPROD                int64
DESCRICAO             object
OBS2                  object
CODAUXILIAR2         float64
CODAUXILIAR          float64
EMBALAGEM             object
EMBALAGEMMASTER       object
QTUNITCX               int64
CODSEC                 int64
SECAO                 object
LASTROPAL              int64
ALTURAPAL              int64
QTTOTPAL               int64
ALTURAM3             float64
LARGURAM3            float64
COMPRIMENTOM3        float64
ALTURAARM            float64
LARGURAARM           float64
COMPRIMENTOARM       float64
PESOBRUTOMASTER      float64
PESOLIQMASTER        float64
PESOBRUTO            float64
PESOLIQ              float64
CAPACIDADE             int64
ABASTECEPALETE        object
USAWMS                object
REVENDA               object
TIPOPROD             float64
CODFORNEC              int64
FORNECEDOR            object
TIPO                  object
PONTOREPOSICAO         int64
RUA                    int64
PREDIO        

Index(['CODFILIAL', 'CODPROD', 'DESCRICAO', 'OBS2', 'CODAUXILIAR2',
       'CODAUXILIAR', 'EMBALAGEM', 'EMBALAGEMMASTER', 'QTUNITCX', 'CODSEC',
       'SECAO', 'LASTROPAL', 'ALTURAPAL', 'QTTOTPAL', 'ALTURAM3', 'LARGURAM3',
       'COMPRIMENTOM3', 'ALTURAARM', 'LARGURAARM', 'COMPRIMENTOARM',
       'PESOBRUTOMASTER', 'PESOLIQMASTER', 'PESOBRUTO', 'PESOLIQ',
       'CAPACIDADE', 'ABASTECEPALETE', 'USAWMS', 'REVENDA', 'TIPOPROD',
       'CODFORNEC', 'FORNECEDOR', 'TIPO', 'PONTOREPOSICAO', 'RUA', 'PREDIO',
       'NIVEL', 'APTO', 'PKESTRU', 'PK_END', 'CARACTERISTICA', 'PULMAO',
       'TIPO_1', 'PRAZOVAL', 'PERCTOLERANCIAVAL', 'DTULTENT'],
      dtype='object')